Visual models are best for people, numerical models are for machines.  Bokeh's visual interactions and static embedding make it the best candidate for data science.  Interacting with information allows the viewer to draw their own unique insights; static visualizations will only tell the creator's story.

This notebook designs a Object Oriented/Functional API for interactive grid plots in Bokeh; this extends a [previous post](https://foreverwhatever.github.io/2017/06/29/GridPlot-In-Bokeh.html).

In [21]:
    import pandas as pd
    from bokeh import plotting, models, layouts, resources
    from operator import gt, lt, eq
    from functools import wraps
    from coffeetools import coffee
    plotting.output_notebook()

    import collections

    from operator import eq, lt, gt

Loading BokehJS ...

In [22]:
    df = pd.util.testing.makeDataFrame().reset_index(drop=True)

* Transform dataframe in ColumnDataSource

In [23]:
    class DeepChainMap(collections.ChainMap):
        def __setitem__(self, key, value):
            for mapping in self.maps:
                if key in mapping:
                    mapping[key] = value
                    return
            self.maps[0][key] = value

        def __delitem__(self, key):
            for mapping in self.maps:
                if key in mapping:
                    del mapping[key]
                    return
            raise KeyError(key)
            
        def new_child(self, m=None):
            return setattr(self, 'maps', super().new_child(m).maps) or self
        
        def complement(self):
            return type(self)(*reversed(self.maps))

In [24]:
    import itertools
    import operator

In [25]:
    def replaceRowCol(obj, **kwargs):
        if isinstance(obj, str):
            obj = obj.format(**kwargs)
        if isinstance(obj, dict):
            obj = {k: replaceRowCol(v, **kwargs) for k, v in obj.items()} 
        return obj

In [26]:
    Figure = lambda **kwargs: plotting.Figure(
            **kwargs, width=200, height=200
        )

In [32]:
    class Plots(object):
        def __init__(self, sources, features=None, figures=None, glyph=None):
            if isinstance(sources, pd.DataFrame):
                sources = DeepChainMap({None: df}).new_child({None: df.pipe(plotting.ColumnDataSource)})
            self.sources = sources
            self.features = features or list(df.columns)
            self.figures = figures
            self.glyph = glyph
            self.figures or self.reset()
            

        def reset(self):
            self._diag, self._upper, self._lower = (
                collections.defaultdict(Figure),
                collections.defaultdict(Figure),
                collections.defaultdict(Figure))
            for cmp, object in zip(
                [eq, gt, lt], operator.attrgetter('_diag', '_lower', '_upper')(self)
            ):
                for row, col in self:
                    cmp(*map(self.features.index, [row, col])) and object[(row, col)]
            self.figures = DeepChainMap(self._diag, self._upper, self._lower)

        def __iter__(self):
            for row, col in itertools.product(*[self.features]*2):
                yield row, col
                
        @property
        def diagonal(self):
            return type(self)(self.sources, self.features, self._diag)
        
        @property
        def upper(self):
            return type(self)(self.sources, self.features, self._upper)
        
        @property
        def lower(self):
            return type(self)(self.sources, self.features, self._lower)
        
        def __getattr__(self, key):
            return setattr(self, 'glyph', getattr(models, key)) or self
        
        def __call__(self, **kwargs):
            for (r, c), p in self.figures.items():
                print(p)
                p.add_glyph(self.sources[None], self.glyph(**replaceRowCol(kwargs, row=r, col=c)))
            return self
        
        def layout(self):
            return layouts.gridplot([self.figures[(r,c)] for r, c in self], ncols=len(self.features))
        
        def show(self):
            return plotting.show(self.layout())

In [34]:
    g = Plots(df)
    g.upper.Circle(x='{row}', y='{col}')
    g.diagonal.Square(x='{row}', y='{col}')
    g.show()

Figure(id='2423d804-95cb-42fb-9d52-61485e832e4e', ...)
Figure(id='f13f6899-1337-4bce-9806-2539cbecc33e', ...)
Figure(id='a7e943e5-c341-421e-868c-209555ae2096', ...)
Figure(id='835dfd8e-9734-4401-a3f6-c602bb9a4e4f', ...)
Figure(id='880839d9-6c31-47a1-9624-bb4065522570', ...)
Figure(id='b5adab30-8340-4d21-ac96-c39a72eaff74', ...)
Figure(id='1dbc3eb5-d958-49be-a8f8-c66448b9dd1d', ...)
Figure(id='2c274500-ea95-4c2f-a07f-70a64407e14c', ...)
Figure(id='aba2713b-5865-4403-a60b-29fa38360736', ...)
Figure(id='08f05c62-4a19-4099-b118-c3c592f9636f', ...)
